# 앙상블 학습(Ensemble Learning)
- 트레이닝 데이터를 기반으로 분류 모형을 여러 개 만들고 서로 비교하는 것이 핵심 아이디어
- 여러 개의 분류기(classifier)을 결합함으로써 개별적인 분류기보다 성능이 뛰어난 최종 분류기를 만드는 것이 주 목적

#####  Q: 앙상블 학습 모형이 개별 모형보다 성능이 뛰어난 이유?

## 보팅 (Voting)
- 여러 개의 분류 모형의 결과를 대상으로 투표를 통해 최종 클래스 라벨을 정하는 방법
- 다수결 투표(plurality voting), 과반수 투표(majority voting)
- 개별 분류기는 여러 가지 알고리즘 사용해서 만들 수 있음: 로지스틱 회귀 ,svm, dt 등
- 개별 분류기들이 동일한 트레이닝 데이터로 학습!

In [1]:
# 데이터 불러오기 (꽃 데이터 분류)
from sklearn import datasets
raw_iris = datasets.load_iris()

# 피처, 타깃 데이터 지정
X = raw_iris.data
y = raw_iris.target

# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [2]:
# 데이터 학습
# 보팅 학습 (여러 가지 머신러닝 모형 이용하는 방법)
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 분석
from sklearn import svm # 서포트 벡터 머신
from sklearn.naive_bayes import GaussianNB # 가우시안 나이브 베이즈
from sklearn.ensemble import VotingClassifier # 보팅을 위한 함수 (회귀 문제라면 VotingRegressor 사용)

clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = svm.SVC(kernel='linear', random_state=1)
clf3 = GaussianNB()

# 세 가지 모형 이용해 보팅 모형 설정
# estimators: 미리 만드느 세 가지 모형 의미
# hard(기본값): 투표 결과로 과반수가 넘는 라벨이 정해지고/soft: 확률이 가장 높은 라벨로 정해짐
# weights: 세 가지 모형의 비율
clf_voting = VotingClassifier(estimators=[
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3)
],
                              voting = 'hard',
                              weights = [1, 1, 1])
clf_voting.fit(X_tn_std, y_tn)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])

In [3]:
# 데이터 예측
pred_voting = clf_voting.predict(X_te_std)
print(pred_voting)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


In [4]:
# 정확도 평가
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)
print(accuracy)

0.9736842105263158


In [5]:
# confusion matrix 확인
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


In [6]:
# 분류 리포트 확인
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



## 배깅과 랜덤 포레스트

### 독립적 앙상블 방법(Independent Ensemble Method)
- 개별 분류기들: 서로 독립적인 알고리즘. 각 분류기는 서로 다른 머신러닝 알고리즘 사용 가능
- 각 분류기가 독립적이므로 효과적으로 병렬화 가능
#### 배깅(bootstrap aggregating)
- 개별 분류기들의 분류 결과 종합하여 최종 분류기의 성능 향상
- 오리지널 트레이닝 데이터 셋에서 부트스트랩(bootstrap) 샘플(중복을 허용한 랜덤 샘플)을 뽑아 학습
- 개별 분류 모형의 결괏값을 모아 다수결 투표(plurality voting)를 통해 최종 예측하게 됨
- 주로 배깅에 사용하는 개별 분류기들은 모두 같은 머신러닝 알고리즘
#### 랜덤 포레스트(random forest)
- 배깅 이용한 가장 유명한 알고리즘
- 여러 개의 개별 분류기인 의사결정나무를 토대로 예측한 결과를 종합해 전체 예측 정확도 높이는 방법
> <랜덤 포레스트 과정>
> - n개의 데이터 랜덤 추출 (중복 가능)
> - p개의 피처 선택 (중복 불가능)
> - 의사 결정 나무 학습
> - (1)-(3) 반복
> - 각 의사 결정 나무별 결과를 투표를 통해 클래스 레이블 설정

##### 랜덤 포레스트 실습

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

raw_wine = datasets.load_wine()

X = raw_wine.data
y = raw_wine.target

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [3]:
# 데이터 학습
from sklearn.ensemble import RandomForestClassifier # 회귀 문제면 RandomForestRegressor 사용
clf_rf = RandomForestClassifier(max_depth=2, random_state=0)
clf_rf.fit(X_tn_std, y_tn)

RandomForestClassifier(max_depth=2, random_state=0)

In [4]:
# 데이터 예측
pred_rf = clf_rf.predict(X_te_std)
print(pred_rf)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


In [5]:
# 정확도 평가
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_rf)
print(accuracy)

0.9555555555555556


In [7]:
# confusion matrix 확인
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_rf)
print(conf_matrix)

[[16  0  0]
 [ 1 19  1]
 [ 0  0  8]]


In [8]:
# 분류 리포트 확인
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_rf)
print(class_report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45



##### 배깅 실습

In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB # 개별 분류기로 사용할 함수
from sklearn.ensemble import BaggingClassifier # 배깅을 위한 함수. 회귀 문제라면 BaggingRegressor 사용

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

raw_wine = datasets.load_wine()

X = raw_wine.data
y = raw_wine.target

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

# best_estimator에 개별 학습기, n_estimators에 개별 분류기 개수
clf_bagging = BaggingClassifier(base_estimator=GaussianNB(), n_estimators=10, random_state=0)
clf_bagging.fit(X_tn_std, y_tn)

pred_clf = clf_bagging.predict(X_te_std)
print(pred_clf)

accuracy = accuracy_score(y_te, pred_clf)
print(accuracy)

conf_matrix = confusion_matrix(y_te, pred_clf)
print(conf_matrix)

class_report = classification_report(y_te, pred_clf)
print(class_report)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]
0.9555555555555556
[[16  0  0]
 [ 1 19  1]
 [ 0  0  8]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45



## 부스팅

### 의존적 앙상블 방법(Dependent Ensemble Method)
- 개별 학습기들이 서로 독립이 아닌 경우

#### 부스팅(Boosting)
- 의존적 앙상블 방법 중 가장 유명한 방법
- 핵심 아이디어: 분류하기 어려운 데이터에 집중
- 약한 학습기(weak learner) 여러 개를 모아 하나의 강한 학습기 만드는 방법
- 관심의 정도 반영. 점차 학습 진행되면서 올바르게 분류된 데이터 포인트의 가중치는 감소, 잘못 분류된 데이터 포인트의 가중치는 증가 (학습이 진행되면서 분류하기 어려운 데이터에 집중)
- 이전 단계에서 만들어진 학습기는 다음 단계에서 사용할 트레이닝 셋의 가중치를 변경하는데 사용
- 배깅과 달리 이전 분류기의 성능에 영향 받음 (새로운 분류기는 이전 분류기의 성능에 따라 잘못 분류된 데이터에 더 집중)
- 각 데이터 포인트에 할당된 가중치에 비례해서 추출

##### 에이다 부스트(AdaBoost)
- 분류하기 어려운 트레이닝 데이터에 가중치를 더 높이는 것 (잘못 분류된 트레이닝 데이터 포인트 가중치 증가)
- 약한 분류기의 결과를 모아 투표 실시하지만, 투표할 때 약한 분류기별로 투표 결과에 가중치 부여
- 일반 부스팅과는 다르게 약한 학습기 훈련할 때 훈련 데이터 셋 전체 사용

In [10]:
# 유방암 여부 예측
from sklearn import datasets
raw_breast_cancer = datasets.load_breast_cancer()

X = raw_breast_cancer.data
y = raw_breast_cancer.target

from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [11]:
from sklearn.ensemble import AdaBoostClassifier # AdaBoostRegressor
clf_ada = AdaBoostClassifier(random_state=0)
clf_ada.fit(X_tn_std, y_tn)

AdaBoostClassifier(random_state=0)

In [12]:
pred_ada = clf_ada.predict(X_te_std)
print(pred_ada)

[0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0]


In [13]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_ada)
print(accuracy)

0.9790209790209791


##### 그래디언트 부스팅 (Gradient Boosting)
- 비용함수를 최적화시킴으로써 학습 능력 향상하는 알고리즘


- "확률적 그래디언트 부스팅(Stochatic Gradient Boosting)">> 사이킷런>GradientBoostingRegressor 실행할 때 subsample 사용하면 각 트리가 학습할 때 사용하느 트레이닝 데이터의 비율 정할 수 있음 (ex. subsample=0.2: 각 트리는 트레이닝 데이터의 20% 비율로 데이터를 랜덤으로 선택)
 
 
- XGBoost
- LightGBM: XGBoost가 무겁다는 단점 보완

In [14]:
# 유방암 여부 예측
from sklearn import datasets
raw_breast_cancer = datasets.load_breast_cancer()

X = raw_breast_cancer.data
y = raw_breast_cancer.target

from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [15]:
# 데이터 학습
from sklearn.ensemble import GradientBoostingClassifier # GradientBoostingRegressor
clf_gbt = GradientBoostingClassifier(max_depth=2, learning_rate=0.01, random_state=0)
clf_gbt.fit(X_tn_std, y_tn)

GradientBoostingClassifier(learning_rate=0.01, max_depth=2, random_state=0)

In [16]:
# 데이터 예측
pred_gboost = clf_gbt.predict(X_te_std)
print(pred_gboost)

[0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 1 1 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0]


In [17]:
# 정확도 평가
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_gboost)
print(accuracy)

0.965034965034965


## 스태킹(Stacking)
- 여러 가지 학습기를 쌓는 방법
- 베이스 학습기(base learner)가 먼저 학습한 후 메타 학습기(meta learner)는 베이스 학습기의 예측을 피처 데이터로 활용해 최종 예측
> <스태킹 알고리즘>
> - (전체데이터 = 트레이닝 셋 + 테스트셋) 중 트레이닝 데이터를 두 개로 분리
> - 베이스 학습기로 첫 번째 트레이닝 데이터 셋 학습
> - 학습된 베이스 학습기에 두 번째 트레이닝 데이터 셋 넣고 예측
> - 예측값을 또 다른 인풋 데이터로 활용해 메타 학습기를 학습

In [18]:
# 유방암 여부 예측
from sklearn import datasets
raw_breast_cancer = datasets.load_breast_cancer()

X = raw_breast_cancer.data
y = raw_breast_cancer.target

from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [21]:
# 데이터 학습
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

clf1 = svm.SVC(kernel='linear', random_state=1)
clf2 = GaussianNB()

# 기본 학습기로 svm, 가우시안 나이브 베이즈 모형 사용
# 메타 학습기로 로지스틱 회귀 분석 사용
# StackingClassifier 이용해서 스태킹 모형 설정 (회귀 문제는 StackingRegressor)
clf_stkg = StackingClassifier(
            estimators=[('svm', clf1),
                       ('gnb', clf2)],
            final_estimator=LogisticRegression())
clf_stkg.fit(X_tn_std, y_tn)

StackingClassifier(estimators=[('svm', SVC(kernel='linear', random_state=1)),
                               ('gnb', GaussianNB())],
                   final_estimator=LogisticRegression())

In [22]:
# 데이터 예측
pred_stkg = clf_stkg.predict(X_te_std)
print(pred_stkg)

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0]


In [23]:
# 정확도 평가
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_stkg)
print(accuracy)

0.965034965034965
